In [1]:
#export
"""
This is for all short utilities that has the boilerplate feeling
"""
from k1lib.bioinfo.cli.init import patchDefaultDelim, BaseCli, settings, Table, T
import k1lib.bioinfo.cli as cli, numbers, torch, numpy as np
from typing import overload, Iterator, Any, List, Set, Union
import k1lib
__all__ = ["size", "shape", "item", "identity",
           "toStr", "to1Str", "toNumpy", "toTensor",
           "toList", "wrapList", "toSet", "toIter", "toRange",
           "equals", "reverse", "ignore",
           "toSum", "toAvg", "toMax", "toMin",
           "lengths", "headerIdx", "deref"]

In [2]:
#export
class size(BaseCli):
    def __init__(self, idx=None):
        """Returns number of rows and columns in the input.
Example::

    # returns (3, 2)
    [[2, 3], [4, 5, 6], [3]] | size()
    # returns 3
    [[2, 3], [4, 5, 6], [3]] | size(0)
    # returns 2
    [[2, 3], [4, 5, 6], [3]] | size(1)
    # returns (2, 0)
    [[], [2, 3]] | size()
    # returns (3, None)
    [2, 3, 5] | size()
    # returns 3
    [2, 3, 5] | size(0)

:param idx: if idx is None return (rows, columns). If 0 or 1, then rows or
    columns"""
        super().__init__(); self.idx = idx
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); it = iter(it)
        if self.idx == 0: # get rows only
            rows = 0
            for line in it: rows += 1
            return rows
        if self.idx == 1: # get #columns only
            return len(next(it))
        columns = -1; rows = 0
        for row in it:
            if columns == -1:
                try: columns = len(list(row))
                except: columns = None
            rows += 1
        if columns == -1: columns = None
        return rows, columns
shape = size

In [3]:
assert [[2, 3], [4, 5, 6], [3]] | size() == (3, 2)
assert [[2, 3], [4, 5, 6], [3]] | size(0) == 3
assert [[2, 3], [4, 5, 6], [3]] | size(1) == 2
assert [[], [2, 3]] | size() == (2, 0)
assert [2, 3, 5] | size() == (3, None)
assert [2, 3, 5] | size(0) == 3

In [4]:
#export
class item(BaseCli):
    """Returns the first row.
Example::

    # returns 0
    iter(range(5)) | item()"""
    def __ror__(self, it:Iterator[str]):
        return next(iter(it))

In [5]:
assert iter(range(5)) | item() == 0

In [6]:
#export
class identity(BaseCli):
    """Yields whatever the input is. Useful for multiple streams.
Example::

    # returns range(5)
    range(5) | identity()"""
    def __ror__(self, it:Iterator[Any]):
        return it

In [7]:
assert range(5) | identity() == range(5)

In [8]:
#export
class toStr(BaseCli):
    def __init__(self, column:int=None):
        """Converts every line to a string.
Example::

    # returns ['2', 'a']
    [2, "a"] | toStr() | deref()
    # returns [[2, 'a'], [3, '5']]
    assert [[2, "a"], [3, 5]] | toStr(1) | deref()"""
        super().__init__(); self.column = column
    def __ror__(self, it:Iterator[str]):
        c = self.column
        if c is None:
            for line in it: yield str(line)
        else:
            for row in it:
                yield [e if i != c else str(e) for i, e in enumerate(row)]

In [9]:
assert [2, "a"] | toStr() | cli.deref() == ['2', 'a']
assert [[2, "a"], [3, 5]] | toStr(1) | cli.deref() == [[2, 'a'], [3, '5']]

In [10]:
#export
class to1Str(BaseCli):
    def __init__(self, delim:str=None):
        r"""Merges all strings into 1, with `delim` in the middle. Basically :meth:`list.join`.
Example::

    # returns '2\na'
    [2, "a"] | to1Str("\n")"""
        super().__init__(); self.delim = patchDefaultDelim(delim)
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); return self.delim.join(it | toStr())

In [11]:
assert [2, "a"] | to1Str("\n") == '2\na'

In [12]:
#export
class toNumpy(BaseCli):
    """Converts generator to numpy array. Essentially ``np.array(list(it))``"""
    def __ror__(self, it:Iterator[float]) -> np.array:
        return np.array(list(it))
class toTensor(BaseCli):
    """Converts generator to :class:`torch.Tensor`. Essentially
``torch.tensor(list(it))``."""
    def __ror__(self, it:Iterator[float]) -> torch.Tensor:
        return torch.tensor(list(it))

In [13]:
#export
class toList(BaseCli):
    """Converts generator to list. :class:`list` would do the
same, but this is just to maintain the style"""
    def __ror__(self, it:Iterator[Any]) -> List[Any]:
        return list(it)
class wrapList(BaseCli):
    """Wraps inputs inside a list"""
    def __ror__(self, it:T) -> List[T]:
        return [it]
class toSet(BaseCli):
    """Converts generator to set. :class:`set` would do the
same, but this is just to maintain the style"""
    def __ror__(self, it:Iterator[T]) -> Set[T]:
        return set(it)
class toIter(BaseCli):
    """Converts object to iterator. `iter()` would do the
same, but this is just to maintain the style"""
    def __ror__(self, it:List[T]) -> Iterator[T]:
        return iter(it)
class toRange(BaseCli):
    """Returns iter(range(len(it))), effectively"""
    def __ror__(self, it:Iterator[Any]) -> Iterator[int]:
        for i, _ in enumerate(it): yield i
class _EarlyExp(Exception): pass
class equals:
    """Checks if all incoming columns/streams are identical"""
    def __ror__(self, streams:Iterator[Iterator[str]]):
        streams = list(streams)
        for row in zip(*streams):
            sampleElem = row[0]
            try:
                for elem in row:
                    if sampleElem != elem: yield False; raise _EarlyExp()
                yield True
            except _EarlyExp: pass
class reverse(BaseCli):
    """Reverses incoming list.
Example::

    # returns [3, 5, 2]
    [2, 5, 3] | reverse() | deref()"""
    def __ror__(self, it:Iterator[str]) -> List[str]:
        return reversed(list(it))

In [14]:
assert [2, 5, 3] | reverse() | cli.deref() == [3, 5, 2]

In [15]:
#export
class ignore(BaseCli):
    r"""Just loops through everything, ignoring the output.
Example::

    # will just return an iterator, and not print anything
    [2, 3] | apply(lambda x: print(x))
    # will prints "2\n3"
    [2, 3] | apply(lambda x: print(x)) | ignore()"""
    def __ror__(self, it:Iterator[Any]):
        for _ in it: pass

In [16]:
with k1lib.CaptureStdout() as out:
    [2, 3] | cli.apply(lambda x: print(x))
assert len(out) == 0
with k1lib.CaptureStdout() as out:
    [2, 3] | cli.apply(lambda x: print(x)) | ignore()
assert out == ["2", "3"]

In [17]:
#export
class toSum(BaseCli):
    """Calculates the sum of list of numbers.
Example::

    # returns 45
    range(10) | toSum()"""
    def __ror__(self, it:Iterator[float]): return sum(it)
class toAvg(BaseCli):
    """Calculates average of list of numbers.
Example::

    # returns 4.5
    range(10) | toAvg()
    # returns nan
    [] | toAvg()"""
    def __ror__(self, it:Iterator[float]):
        s = 0; i = -1
        for i, v in enumerate(it): s += v
        i += 1
        if not settings["strict"] and i == 0: return float("nan")
        return s / i

In [18]:
assert range(10) | toSum() == 45
assert range(10) | toAvg() == 4.5
assert id([] | toAvg()) == id(float("nan"))

In [19]:
#export
class toMax(BaseCli):
    """Calculates the max of a bunch of numbers.
Example::

    # returns 6
    [2, 5, 6, 1, 2] | toMax()"""
    def __ror__(self, it:Iterator[float]) -> float: return max(it)
class toMin(BaseCli):
    """Calculates the min of a bunch of numbers.
Example::

    # returns 1
    [2, 5, 6, 1, 2] | toMin()"""
    def __ror__(self, it:Iterator[float]) -> float: return min(it)

In [20]:
assert [2, 5, 6, 1, 2] | toMax() == 6
assert [2, 5, 6, 1, 2] | toMin() == 1

In [21]:
#export
class lengths(BaseCli):
    """Returns the lengths of each row.
Example::

    [range(5), range(10)] | lengths() == [5, 10]"""
    def __ror__(self, it:Iterator[List[Any]]) -> Iterator[int]:
        for e in it: yield len(e)

In [22]:
assert [range(5), range(10)] | lengths() | cli.deref() == [5, 10]

In [23]:
#export
def headerIdx():
    """Cuts out first line, put an index column next to it, and prints it
out. Useful when you want to know what your column's index is to cut it
out. Also sets the context variable "header", in case you need it later.
Example::

    # returns [[0, 'a'], [1, 'b'], [2, 'c']]
    ["abc"] | headerIdx() | deref()
    # returns ['a', 'b', 'c']
    ctx["header"]()"""
    return item() | cli.ctx.consume("header") | cli.tableFromList() | cli.insertIdColumn(True)

In [24]:
assert ["abc"] | headerIdx() | cli.deref() == [[0, 'a'], [1, 'b'], [2, 'c']]
assert cli.ctx["header"]() == ['a', 'b', 'c']

In [25]:
#export
Number = numbers.Number; Tensor = torch.Tensor; NpNumber = np.number
class inv_dereference(BaseCli):
    def __init__(self, ignoreTensors=False):
        """Kinda the inverse to :class:`dereference`"""
        super().__init__(); self.ignoreTensors = ignoreTensors
    def __ror__(self, it:Iterator[Any]) -> List[Any]:
        super().__ror__(it); ignoreTensors = self.ignoreTensors; 
        for e in it:
            if isinstance(e, cli.ctx.Promise): e = e()
            if e is None or isinstance(e, (Number, NpNumber, str)): yield e
            elif isinstance(e, Tensor):
                if not ignoreTensors and len(e.shape) == 0: yield e.item()
                else: yield e
            else:
                try: yield e | self
                except: yield e
class deref(BaseCli):
    def __init__(self, ignoreTensors=False, maxDepth=float("inf")):
        """Recursively converts any iterator into a list. Only :class:`str`,
:class:`numbers.Number` are not converted. Example::

    # returns something like "<range_iterator at 0x7fa8c52ca870>"
    iter(range(5))
    # returns [0, 1, 2, 3, 4]
    iter(range(5)) | deference()

You can also specify a ``maxDepth``::

    # returns something like "<list_iterator at 0x7f810cf0fdc0>"
    iter([range(3)]) | deref(maxDepth=0)
    # returns [range(3)]
    iter([range(3)]) | deref(maxDepth=1)
    # returns [[0, 1, 2]]
    iter([range(3)]) | deref(maxDepth=2)

:param ignoreTensors: if True, then don't loop over :class:`torch.Tensor`
    internals
:param maxDepth: maximum depth to dereference. Starts at 0 for not doing anything
    at all

.. warning::

    Can work well with PyTorch Tensors, but not Numpy's array as they screw things up
    with the __ror__ operator, so do torch.from_numpy(...) first. Don't worry about
    unnecessary copying, as numpy and torch both utilizes the buffer protocol."""
        super().__init__(); self.ignoreTensors = ignoreTensors
        self.maxDepth = maxDepth; self.depth = 0
    def __ror__(self, it:Iterator[T]) -> List[T]:
        super().__ror__(it); answer = []; ignoreTensors = self.ignoreTensors
        if self.depth >= self.maxDepth: return it
        self.depth += 1
        for e in it:
            if isinstance(e, cli.ctx.Promise): e = e()
            if e is None or isinstance(e, (Number, NpNumber, str)):
                answer.append(e)
            elif isinstance(e, Tensor):
                if not ignoreTensors and len(e.shape) == 0:
                    answer.append(e.item())
                else: answer.append(e)
            else:
                try: answer.append(e | self)
                except: answer.append(e)
        self.depth -= 1
        return answer
    def __invert__(self) -> BaseCli:
        """Returns a :class:`~k1lib.bioinfo.cli.init.BaseCli` that makes
everything an iterator."""
        return inv_dereference(self.ignoreTensors)

In [26]:
import numpy as np, numbers, torch
a = torch.linspace(0, 10, 50) | deref()
b = torch.from_numpy(np.linspace(0, 10, 50)) | deref()
assert torch.allclose(torch.tensor(b), torch.tensor(a))

assert iter([range(3)]) | deref(maxDepth=1) == [range(3)]
assert iter([range(3)]) | deref(maxDepth=2) == [[0, 1, 2]]

In [28]:
!../../../export.py bioinfo/cli/utils

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.13
Uninstalling k1lib-0.1.13:
  Successfully uninstalled k1lib-0.1.13
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1